# Create Vector Data

ScrollStats uses the following vector datasets for every bend in order to calculate ridge metrics:
- Channel Centerline (LineString)
- Bend Boundary (Polygon)
- Packet Boundaries (Polygon)
- Ridges (LineString)
- Migration Pathways (LineString)

While all of these datasets are used, not all are strictly necesarry, not all are required at the beginning of the workflow, and some are generated from ScrollStats. 

This is summarized below:

**Required for Raster Processing**:
- Bend Boundary: the bend boundary is a required input from the user and is required for the raster processing. The bend boundary is the manual delineation of the topographically rasied platform that conatians the ridge and swale topography of a bend.

**Required for Migration Pathway Creation**
- Channel Centerline: the channel centerline generally follows the path of the water surface and should not intersect the bend boundary at all. The channel centerline should also extend past the channel-ward edges of the bend boundary. *See note at the end of this cell on point density for channel centerline and ridge lines.*

- Ridges: ridges are manually created and should track the middle of each ridge area. While it is not necesarry, it's strongly reccommended that both the DEM and ridge area raster be used to infer ridge line locations. *See note at the end of this cell on point density for channel centerline and ridge lines.*

**Used for ridge metric calculation**
- Migration Pathways: migration pathways are created by ScrollStats, so the user does not need to worry about creating these as an input. Migration pathways are a series of transects that trace the supposed path of channel migration from the channel centerline. Migration pathways are used to sample both the DEM and ridge area raster to calculate ridge width, amplitude, and spacing for every point intersection of the migration pathways and ridge lines.

- Packet Boundaries (optional): packet boundaries are the manual delineations of the depositional packets within a bend that contain groups of ridges with similar trajectorties. Packet boundaries fit perfectly within and cover entirely the bend boundary. Providing packet boundaries gives allows the user to aggregate ridge metrics to the packet scale in order to investigate changes in the hydrologic regime of the river.

**Note on point density for channel centerline and ridgelines**

A high vertex density and high degree of "smoothness" are necesarry for both the channel centerline and ridge lines when creating the migration pathways. It is reccommended that you focus on capturing the overall ridge form, not point density, when delineating. Then, the coarse polylines can be densified and smoothed with the included [line_smoother.py](scrollstats/delineation/line_smoother.py) script. 


In [ ]:
from scrollstats import LineSmoother
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

from parameters import SMOOTHING_WINDOW_SIZE, VERTEX_SPACING, SHOOT_DISTANCE, SEARCH_DISTANCE, DEV_FROM_90
from scrollstats import create_transects

## Create Bend Boundaries

Use a desktop GIS software (QGIS, ArcGIS Pro, etc.) to manually delineate the bend boundaries.

If there multiple bends delineated for the same river, it is reccommended to make 1 layer for the entire river and just add each bend as a feature (row) for that layer. Additionally, each bend should have a short, unique ID that is shared with the DEMs. For example, the 25th bend along the Lower Brazos River (starting from Waco TX) is given the ID `LBR_025`.

At this stage, it is reccommended to create the optional packet boundaries as well. Similarly to the bend boundaries, it is reccommended to make one layer for all the packets in the river and just add each packet as a feature (row). Add a "bend_id" column and a "packet_id" column. The bend id should correspond to the bend_id of the encompassing boundary and it is reccommended to have use a simple incrementing packet id that corresponds to the apparent evolution of the bend. For example, the most ancestral packet of `LBR_025` has the packet_id of `p_01` and the most recent, channelward packet has a packet_id of `p_04`. The combination of the bend_id and packet_id column forms the primary key for the packets. 

Example bend and packet boundaries can be found in the [example_data](example_data) directory. These boundaries are mapped below.

In [ ]:
bend_area = gpd.read_file("example_data/input/LBR_025_bend.geojson").set_index("bend_id")
packets = gpd.read_file("example_data/input/LBR_025_packets.geojson").set_index("packet_id")

fig, ax = plt.subplots(1,1, figsize=(10, 6))

packets.boundary.plot(color="grey", ax=ax, label="Packet Boundaries")
packets.plot(color="grey", alpha=0.5, ax=ax)
bend_area.boundary.plot(color="k", lw=3, ax=ax, label="Bend Boundary")

ax.legend(loc="upper left")
ax.set_axis_off()

## Create Channel Centerline and Ridgelines
After the ridge area rasters are created, manually create the ridge lines and channel centerlines.

Example centerline and ridgelines can be found in the [example_data](example_data) directory. The centerline and ridgelines are mapped below.

In [ ]:
ridge_path = Path("example_data/input/LBR_025_ridges_manual.geojson")
manual_ridges = gpd.read_file(ridge_path)

# Centerline is already smoothed and densified
cl_path = Path("example_data/input/LBR_025_cl.geojson")
cl = gpd.read_file(cl_path)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 6))

manual_ridges.plot(color="k", ls=":", ax=ax, label="Ridges (Manual)")
cl.plot(color="k", lw=2, ax=ax, label="Centerline")

ax.legend(loc="upper left")
ax.set_axis_off()

### Line Smoothing

Apply `LineSmoother` to smooth and densify the lines

In [ ]:
window = SMOOTHING_WINDOW_SIZE
spacing = VERTEX_SPACING

# Smooth and densify the lines
ls = LineSmoother(manual_ridges, spacing, window)
smooth_ridges = ls.execute()

# Save smooth ridges to disk
output_dir = Path("example_data/output")
smooth_ridge_name = ridge_path.with_stem(ridge_path.stem + "_smoothed").name
smooth_ridge_path = output_dir / smooth_ridge_name

smooth_ridges.to_file(smooth_ridge_path, driver="GeoJSON", index=False)

# Plot manual and smoothed lines for comparison
m = manual_ridges.explore(color="black", style_kwds={"weight":5})
smooth_ridges.explore(color="red", m=m)

## Create Migration Pathways

Now that the ridges and centerlines are created, we can create the migration pathways

A migration pathway can be created from any point along the centerline. Below we will create an array of starting points each roughly a channel-width apart (~100m for the Lower Brazos). 

You can also create a set number of starting points by defining your `step` distance as the channel length divided by the number of evenly spaced transects. 

In [ ]:
# define the distance between transects 
step = 100

# With a vertex spacing of ~1m, take every `step`th vertex along the centerline
starts = np.asarray(cl.geometry[0].xy).T[::step]

### Migration Pathway Algorithm
Our process involves ten major steps as follows:
1.	Let the channel centerline be called R1
2.	Select a starting point on R1, let this point be called P1
3.	From P1, shoot a given distance perpendicular from R1 in the direction of the convex bank.
4.	If the line intersects a ridge line, call this ridge R2 and proceed to step 5. If not, move on to the next location on R1 and repeat step 2.
5.	Where this new line intersects R2, make a new point and call it N1
6.	Buffer N1 by a given radius and search all vertices of R2 that intersect this buffer for a point from which a line may be drawn back to P1 that is perpendicular to R2. Call this new point on R2, N2.
7.	Treating the lines P1->N1 and P1->N2 as vectors, calculate their vertical resultant and place a new point at the end of this vertical resultant. Let this point be called VR. 
8.	Where the line P1->VR intersects R2, place a point and let this point be called P2. Line P1->P2 is the migration pathway for this location on the floodplain. 
9.	Redefine P2 as P1
10.	Repeat steps 3-9 until the perpendicular shot from P1 fails to intersect any ridges.

![Alt text](image.png)

In [ ]:
# Transect Parameters
shoot_distance = SHOOT_DISTANCE       # Distance that the N1 coordinate will shoot out from point P1; easured in linear unit of dataset
search_distance = SEARCH_DISTANCE     # Buffer radius used to search for an N2 coordinate on R2; measured in linear unit of dataset
dev_from_90 = DEV_FROM_90             # Max angular deviation from 90° allowed when searching for an N2 coordinate on R2; measured in degrees

transects = create_transects(cl, smooth_ridges, step, shoot_distance, search_distance, dev_from_90)

# Save transects to disk
transect_path = output_dir / "LBR_025_transects.geojson"
transects.to_file(transect_path, dirver="GeoJSON", index=True)

In [ ]:
# Plot migration pathways alongside ridges and centerlines 
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

cl.plot(color="k", lw=2, ax=ax, label="Centerline")
smooth_ridges.plot(color="k", ls=":", ax=ax, label="Ridges")
transects.plot(color="r", ax=ax, label="Migration Pathways")
plt.scatter(starts[:, ::2], starts[:, 1::2], color="r")

ax.legend()
ax.set_axis_off()

## Move on to CalculateRidgeMetrics.ipynb
- All of the required datasets for ridge metric calculation have now been created. 
- Open the notebook `CalculateRidgeMetrics.ipynb` to calculate ridge metrics.